In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("attribution_allocation_student_data.csv")

In [3]:
df

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3
68120,False,referral,referral,referral,referral,direct,3
68121,False,display,referral,social,direct,NaN,3
68122,False,email,email,social,direct,NaN,3


In [6]:
df.tier.value_counts()

3    34062
2    22708
1    11354
Name: tier, dtype: int64

In [7]:
df[df.convert_TF == True].tier.value_counts()

3    6063
2    4892
1    2824
Name: tier, dtype: int64

In [11]:
percentage1 = len(df[df.tier == 1][df.convert_TF == True]) / len(df[df.tier == 1])
print(percentage1)
percentage2 = len(df[df.tier == 2][df.convert_TF == True]) / len(df[df.tier == 2])
print(percentage2)
percentage3 = len(df[df.tier == 3][df.convert_TF == True]) / len(df[df.tier == 3])
print(percentage3)

0.24872291703364452
0.21543068522106745
0.17799894310375197


# Part 1: attribution

In [12]:
# Average CAC of each tier 
tier_3_cost = 15000/ len(df[df.tier == 3][df.convert_TF == True])
tier_2_cost = 10000/ len(df[df.tier == 2][df.convert_TF == True])
tier_1_cost = 5000/ len(df[df.tier == 1][df.convert_TF == True])

In [14]:
print(tier_1_cost)
print(tier_2_cost)
print(tier_3_cost)

1.7705382436260624
2.044153720359771
2.4740227610094014


## method 1: last interaction

In [15]:
df["last_interact"] = df.touch5

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,last_interact
0,True,referral,referral,referral,email,NaN,1,NaN
1,True,referral,display,display,email,NaN,1,NaN
2,True,email,display,referral,NaN,NaN,1,NaN
3,True,referral,referral,email,NaN,NaN,1,NaN
4,True,social,referral,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3,NaN
68120,False,referral,referral,referral,referral,direct,3,direct
68121,False,display,referral,social,direct,NaN,3,NaN
68122,False,email,email,social,direct,NaN,3,NaN


In [16]:
df.last_interact[df.last_interact.isna()] = df[df.touch5.isna()].touch4
df.last_interact[df.last_interact.isna()] = df[df.touch5.isna()].touch3
df.last_interact[df.last_interact.isna()] = df[df.touch5.isna()].touch2
df.last_interact[df.last_interact.isna()] = df[df.touch5.isna()].touch1

In [17]:
df

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,last_interact
0,True,referral,referral,referral,email,NaN,1,email
1,True,referral,display,display,email,NaN,1,email
2,True,email,display,referral,NaN,NaN,1,referral
3,True,referral,referral,email,NaN,NaN,1,email
4,True,social,referral,NaN,NaN,NaN,1,referral
...,...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3,direct
68120,False,referral,referral,referral,referral,direct,3,direct
68121,False,display,referral,social,direct,NaN,3,direct
68122,False,email,email,social,direct,NaN,3,direct


In [36]:
cost_emial = 1000 + 2000 + 3000
cost_social = 1000 + 2000 + 3000
cost_display = 1000 + 2000 + 3000
cost_paid_search = 1000 + 2000 + 3000
cost_referral = 1000 + 2000 + 3000
cost_organic_search = 0
cost_direct = 0
CAC_email = cost_emial / len(df[df.convert_TF == True][df.last_interact == 'email'])
print('Email: %.2f'%CAC_email)
CAC_social = cost_social / len(df[df.convert_TF == True][df.last_interact == 'social'])
print('Social: %.2f'%CAC_social)
CAC_display = cost_display / len(df[df.convert_TF == True][df.last_interact == 'display'])
print('Display: %.2f'%CAC_display)
CAC_paid_search = cost_paid_search / len(df[df.convert_TF == True][df.last_interact == 'paid_search'])
print('Paid_search: %.2f'%CAC_paid_search)
CAC_referral = cost_referral / len(df[df.convert_TF == True][df.last_interact == 'referral'])
print('Referral: %.2f'%CAC_referral)

Email: 4.02
Social: 2.03
Display: 2.62
Paid_search: 5.98
Referral: 1.00


In [42]:
#check
len(df[df.convert_TF == True][df.last_interact == 'referral']) + len(df[df.convert_TF == True][df.last_interact == 'paid_search']) + len(df[df.convert_TF == True][df.last_interact == 'display'])+len(df[df.convert_TF == True][df.last_interact == 'social'])+len(df[df.convert_TF == True][df.last_interact == 'email'])+len(df[df.convert_TF == True][df.last_interact == 'direct'])+len(df[df.convert_TF == True][df.last_interact == 'organic_search'])

13779

## method 2: first interaction

In [38]:
df["first_interact"] = df.touch1
df

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,last_interact,first_interact
0,True,referral,referral,referral,email,NaN,1,email,referral
1,True,referral,display,display,email,NaN,1,email,referral
2,True,email,display,referral,NaN,NaN,1,referral,email
3,True,referral,referral,email,NaN,NaN,1,email,referral
4,True,social,referral,NaN,NaN,NaN,1,referral,social
...,...,...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3,direct,social
68120,False,referral,referral,referral,referral,direct,3,direct,referral
68121,False,display,referral,social,direct,NaN,3,direct,display
68122,False,email,email,social,direct,NaN,3,direct,email


In [39]:
cost_emial = 1000 + 2000 + 3000
cost_social = 1000 + 2000 + 3000
cost_display = 1000 + 2000 + 3000
cost_paid_search = 1000 + 2000 + 3000
cost_referral = 1000 + 2000 + 3000
cost_organic_search = 0
cost_direct = 0
CAC_email = cost_emial / len(df[df.convert_TF == True][df.first_interact == 'email'])
print('Email: %.2f'%CAC_email)
CAC_social = cost_social / len(df[df.convert_TF == True][df.first_interact == 'social'])
print('Social: %.2f'%CAC_social)
CAC_display = cost_display / len(df[df.convert_TF == True][df.first_interact == 'display'])
print('Display: %.2f'%CAC_display)
CAC_paid_search = cost_paid_search / len(df[df.convert_TF == True][df.first_interact == 'paid_search'])
print('Paid_search: %.2f'%CAC_paid_search)
CAC_referral = cost_referral / len(df[df.convert_TF == True][df.first_interact == 'referral'])
print('Referral: %.2f'%CAC_referral)

Email: 4.67
Social: 2.63
Display: 2.95
Paid_search: 8.98
Referral: 0.80


In [41]:
#check
len(df[df.convert_TF == True][df.first_interact == 'referral']) + len(df[df.convert_TF == True][df.first_interact == 'paid_search']) + len(df[df.convert_TF == True][df.first_interact == 'display'])+len(df[df.convert_TF == True][df.first_interact == 'social'])+len(df[df.convert_TF == True][df.first_interact == 'email'])+len(df[df.convert_TF == True][df.first_interact == 'direct'])+len(df[df.convert_TF == True][df.first_interact == 'organic_search'])

13779

## method 3: linear model

In [47]:
df = df.drop(columns = ['last_interact','first_interact'])

In [60]:
df['email'] = (df['touch1'] == 'email').astype('int') + (df['touch2'] == 'email').astype('int') + (df['touch3'] == 'email').astype('int') + (df['touch4'] == 'email').astype('int') + (df['touch5'] == 'email').astype('int')
df['social'] = (df['touch1'] == 'social').astype('int') + (df['touch2'] == 'social').astype('int') + (df['touch3'] == 'social').astype('int') + (df['touch4'] == 'social').astype('int') + (df['touch5'] == 'social').astype('int')
df['display'] = (df['touch1'] == 'display').astype('int') + (df['touch2'] == 'display').astype('int') + (df['touch3'] == 'display').astype('int') + (df['touch4'] == 'display').astype('int') + (df['touch5'] == 'display').astype('int')
df['paid_search'] = (df['touch1'] == 'paid_search').astype('int') + (df['touch2'] == 'paid_search').astype('int') + (df['touch3'] == 'paid_search').astype('int') + (df['touch4'] == 'paid_search').astype('int') + (df['touch5'] == 'paid_search').astype('int')
df['referral'] = (df['touch1'] == 'referral').astype('int') + (df['touch2'] == 'referral').astype('int') + (df['touch3'] == 'referral').astype('int') + (df['touch4'] == 'referral').astype('int') + (df['touch5'] == 'referral').astype('int')
df['direct'] = (df['touch1'] == 'direct').astype('int') + (df['touch2'] == 'direct').astype('int') + (df['touch3'] == 'direct').astype('int') + (df['touch4'] == 'direct').astype('int') + (df['touch5'] == 'direct').astype('int')
df['organic_search'] = (df['touch1'] == 'organic_search').astype('int') + (df['touch2'] == 'organic_search').astype('int') + (df['touch3'] == 'organic_search').astype('int') + (df['touch4'] == 'organic_search').astype('int') + (df['touch5'] == 'organic_search').astype('int')

In [62]:
df_convert = df[df.convert_TF == True]

In [66]:
df_convert['total'] = df_convert['email'] + df_convert['social'] + df_convert['display'] + df_convert['paid_search'] + df_convert['referral'] + df_convert['direct'] + df_convert['organic_search']

In [67]:
df_convert

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,email,social,display,paid_search,referral,direct,organic_search,total
0,True,referral,referral,referral,email,NaN,1,1,0,0,0,3,0,0,4
1,True,referral,display,display,email,NaN,1,1,0,2,0,1,0,0,4
2,True,email,display,referral,NaN,NaN,1,1,0,1,0,1,0,0,3
3,True,referral,referral,email,NaN,NaN,1,1,0,0,0,2,0,0,3
4,True,social,referral,NaN,NaN,NaN,1,0,1,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60648,True,referral,referral,referral,NaN,NaN,3,0,0,0,0,3,0,0,3
60649,True,display,referral,NaN,NaN,NaN,3,0,0,1,0,1,0,0,2
68103,True,organic_search,referral,paid_search,NaN,NaN,3,0,0,0,1,1,0,1,3
68104,True,organic_search,display,NaN,NaN,NaN,3,0,0,1,0,0,0,1,2


In [68]:
for a in ['email', 'social', 'display', 'paid_search', 'referral', 'direct', 'organic_search']:
    df_convert[a] = df_convert[a] / df_convert['total']

In [70]:
df_convert

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,email,social,display,paid_search,referral,direct,organic_search,total
0,True,referral,referral,referral,email,NaN,1,0.250000,0.0,0.000000,0.000000,0.750000,0.0,0.000000,4
1,True,referral,display,display,email,NaN,1,0.250000,0.0,0.500000,0.000000,0.250000,0.0,0.000000,4
2,True,email,display,referral,NaN,NaN,1,0.333333,0.0,0.333333,0.000000,0.333333,0.0,0.000000,3
3,True,referral,referral,email,NaN,NaN,1,0.333333,0.0,0.000000,0.000000,0.666667,0.0,0.000000,3
4,True,social,referral,NaN,NaN,NaN,1,0.000000,0.5,0.000000,0.000000,0.500000,0.0,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60648,True,referral,referral,referral,NaN,NaN,3,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,3
60649,True,display,referral,NaN,NaN,NaN,3,0.000000,0.0,0.500000,0.000000,0.500000,0.0,0.000000,2
68103,True,organic_search,referral,paid_search,NaN,NaN,3,0.000000,0.0,0.000000,0.333333,0.333333,0.0,0.333333,3
68104,True,organic_search,display,NaN,NaN,NaN,3,0.000000,0.0,0.500000,0.000000,0.000000,0.0,0.500000,2


In [72]:
cost_emial = 1000 + 2000 + 3000
cost_social = 1000 + 2000 + 3000
cost_display = 1000 + 2000 + 3000
cost_paid_search = 1000 + 2000 + 3000
cost_referral = 1000 + 2000 + 3000
cost_organic_search = 0
cost_direct = 0
CAC_email = cost_emial / df_convert.email.sum()
print('Email: %.2f'%CAC_email)
CAC_social = cost_social / df_convert.social.sum()
print('Social: %.2f'%CAC_social)
CAC_display = cost_display / df_convert.display.sum()
print('Display: %.2f'%CAC_display)
CAC_paid_search = cost_paid_search / df_convert.paid_search.sum()
print('Paid_search: %.2f'%CAC_paid_search)
CAC_referral = cost_referral / df_convert.referral.sum()
print('Referral: %.2f'%CAC_referral)

Email: 4.71
Social: 2.61
Display: 2.99
Paid_search: 8.51
Referral: 0.80


In [73]:
#check
df_convert.email.sum() + df_convert.social.sum() + df_convert.display.sum() + df_convert.paid_search.sum() + df_convert.referral.sum() + df_convert.direct.sum() + df_convert.organic_search.sum()

13779.0

# Part 2 : linear method

In [83]:
for a in ['email', 'social', 'display', 'paid_search', 'referral']:
    first_1000_cac = 1000 / df_convert[df.tier == 1][a].sum()
    second_1000_cac = 1000 / (df_convert[df.tier == 2][a].sum() - df_convert[df.tier == 1][a].sum())
    third_1000_cac = 1000 / (df_convert[df.tier == 3][a].sum() - df_convert[df.tier == 2][a].sum())
    print(a, first_1000_cac, second_1000_cac, third_1000_cac)

email 3.9819485001327317 5.225115387964817 7.197696737044143
social 2.063628546861565 2.9604776237232944 5.784805244890093
display 2.276521475185916 3.5110304874480667 8.194482381862882
paid_search 7.809449433814917 9.110233829334952 9.883050568275408
referral 0.659152330103487 0.8775137111517366 1.5788642703015627


Allocation : 
referral 3000
social 3000
display 2000
email 3000